# Connect to controlHub

In [ ]:
import os
import sys

sys.path.append("./libs")
import json
import random
import uuid
import warnings
from os import name
from random import *
from threading import Condition

import jobCtrl as jc
import streamsets
import streamsets.sdk
import utilslib as utl
from streamsets.sdk import *
from streamsets.sdk.config import *
from streamsets.sdk.constants import *
from streamsets.sdk.examples import *
from streamsets.sdk.exceptions import *
from streamsets.sdk.models import *
from streamsets.sdk.sch import *
from streamsets.sdk.sch_api import *
from streamsets.sdk.sch_models import *
from streamsets.sdk.sdc import *
from streamsets.sdk.sdc_models import *
from streamsets.sdk.utils import *

warnings.filterwarnings("ignore")

# ------------------------------------------------
cfg = utl.Config("./config_file_streamsets_sch_trailer.yaml")
cfg.load()
# ------------------------------------------------
os.environ["STREAMSETS_SDK_ACTIVATION_KEY"] = cfg.getSdkKey()
_BASE_URL_ = cfg.getUrlSch()
print(_BASE_URL_)
_SDC_URL_ = cfg.getUrlSdc()
print(_SDC_URL_)
_TX_URL_ = cfg.getUrlTx()
print(_TX_URL_)

# ------------------------------------------------
# ------------------------------------------------
# ------------------------------------------------

ControlHub.VERIFY_SSL_CERTIFICATES = False
DataCollector.VERIFY_SSL_CERTIFICATES = False
Transformer.VERIFY_SSL_CERTIFICATES = False
control_hub = ControlHub(
    cfg.getUrlSch(), username=cfg.getUser(), password=cfg.getPassword()
)
sdc = DataCollector(cfg.getUrlSdc(), control_hub=control_hub)
sdc_authoring = control_hub.data_collectors.get(url=cfg.getUrlSdc())
tx = control_hub.transformers.get(url=cfg.getUrlTx())

# Create a fragments

In [ ]:
for i in range(1, 4):
    pb = control_hub.get_pipeline_builder(fragment=True, data_collector=sdc_authoring)
    s0 = pb.add_stage("Jython Evaluator")
    s0.script = "import os"

    s0.script = (
        " 1 - substring stmt per column that you want to infer from the input dataframe"
    )

    #
    s1 = pb.add_stage("Jython Evaluator")
    s1.script = "# push by sdk"

    s2 = pb.add_stage("Field Remover")
    #     s1.action = "REMOVE"
    # #
    s0 >> s1 >> s2
    #
    pipeline = pb.build(title=f"frag_demo_ut_{i}")
    _ = control_hub.publish_pipeline(pipeline)
    print(f"frag_demo_ut_{i}")

## Extract fragments

In [ ]:
fragments = [
    frag
    for frag in control_hub.pipelines.get_all(fragment=True)
    if frag.name.startswith("frag_demo_ut")
]
fragments

# Create pipelines to wrap fragments 

In [ ]:
for fragment in fragments:
    pb = control_hub.get_pipeline_builder(fragment=False, data_collector=sdc_authoring)
    s0 = pb.add_stage("Azure Data Lake Storage Gen2", type="origin")
    s0.account_fqdn = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
    s0.secure_connection = True
    s0.authentication_method = "Shared Key"
    s0.account_shared_key = "XXXXXXXXXXXXXXXXXXXXXXXXXXXX"
    # setattr(s0,"storage_container_/_file_system","XXXXXXXXXXX")
    s0.configuration.update(
        {
            "dataLakeGen2SourceConfigBean.dataLakeConfig.connection.storageContainer": "XXXXXXXXXXXXXXXXX"
        }
    )
    s1 = pb.add_fragment(fragment)
    s2 = pb.add_stage("Trash")
    _ = s0 >> s1 >> s2
    pipeline = pb.build(title=f"pipe_{fragment.name}")
    _ = control_hub.publish_pipeline(pipeline)
    print(f"pipe_{fragment.name}")

# Create jobs to execute pipelines - fragments

In [ ]:
pipes = [
    pipe for pipe in control_hub.pipelines.get_all() if pipe.name.startswith("pipe_")
]
pipes

In [ ]:
for pipe in pipes:
    job_builder = control_hub.get_job_builder()
    jn = f"job_{pipe.name}"
    print(jn)

    try:
        jd = control_hub.jobs.get(job_name=jn)
        control_hub.delete_job(jd)
    except:
        pass

    p = control_hub.pipelines.get(name=pipe.name)
    prms = {"p1": "val1", "p2": "val2"}

    job = job_builder.build(jn, pipeline=p, runtime_parameters=prms, tags=["demo/ut"])
    job.enable_time_series_analysis = True
    job.enable_failover = True
    job.description = jn
    job.data_collector_labels = ["pool_01"]
    control_hub.add_job(job)